# 通し課題 模範解答（分類編）DAY 3
- kaggle の [Kiva Crowdfunding](https://www.kaggle.com/datasets/kiva/data-science-for-good-kiva-crowdfunding?resource=download) を一部改変したデータを使用
- 「融資予定だった資金が全額集まったか、集まらなかったか」を予測するモデルを作成

## DAY 2（第 4 章～第 7 章）との違い 
- ロジスティック回帰ではなく、ランダムフォレストのモデルを作成する
- 交差検証を用いつつ、ハイパーパラメータ探索（グリッドサーチ）を行う

## DAY 3（第 8 章～第 10 章）で扱う内容
- 前処理後のデータを読み込み
- ハイパーパラメータ探索
  - モデルの構築（ランダムフォレスト）
  - グリッドサーチと交差検証法
  - モデルの評価

## 1. ライブラリの読み込み

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# pkl ファイルの入出力
import pickle

# ランダムフォレストを用いた分類モデル
from sklearn.ensemble import RandomForestClassifier
# 交差検証法とグリッドサーチ
from sklearn.model_selection import KFold, GridSearchCV

# 分類問題の評価指標
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 時刻の取得（処理時間の計測）
import time

## 2. 前処理後のデータを読み込み
DAY 2 の模範回答で前処理を適用したデータが `day3/data/data_classification.pkl` に保存されている

### 2-1. pkl ファイルの読み込み

In [2]:
# pkl ファイルの中身を dict 型の変数として読み込み
with open('../data/data_classification.pkl', 'rb') as f:
    data = pickle.load(f)

# データの形状を確認
for k, d in data.items():
    print(f'{k}: {d.shape}')
    
# 配列に分解
X_train, y_train, X_test, y_test = \
    data['X_train'], data['y_train'], data['X_test'], data['y_test']

X_train: (60894, 84)
y_train: (60894,)
X_test: (29499, 84)
y_test: (29499,)


### 2-2. データの確認

In [3]:
X_train.head()

,loan_amount,term_in_months,sector_Agriculture,sector_Arts,sector_Clothing,sector_Construction,sector_Education,sector_Entertainment,sector_Food,sector_Health,...,currency_XAF,currency_XCD,currency_XOF,currency_YER,currency_ZAR,currency_ZMW,repayment_interval_bullet,repayment_interval_irregular,repayment_interval_monthly,repayment_interval_weekly
51893,0.064802,-0.068545,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
49251,-0.972867,-0.068545,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
85690,2.375974,0.473673,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4893,-0.171032,-0.068545,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
89919,-0.689866,-0.068545,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
X_test.head()

,loan_amount,term_in_months,sector_Agriculture,sector_Arts,sector_Clothing,sector_Construction,sector_Education,sector_Entertainment,sector_Food,sector_Health,...,currency_XAF,currency_XCD,currency_XOF,currency_YER,currency_ZAR,currency_ZMW,repayment_interval_bullet,repayment_interval_irregular,repayment_interval_monthly,repayment_interval_weekly
51174,-0.972867,-1.152981,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4891,1.385472,-0.068545,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
31353,2.328807,4.630675,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
90949,1.385472,2.100326,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
42526,-0.689866,-0.068545,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
y_train.head()

51893    0
49251    1
85690    0
4893     0
89919    1
Name: funded_state, dtype: int64

In [6]:
y_test.head()

51174    1
4891     0
31353    1
90949    0
42526    0
Name: funded_state, dtype: int64

## 3. ハイパーパラメータ探索

### 3-1. モデルの構築
- 以下の 2 つのハイパーパラメータをチューニングしつつ学習する
    - n_estimators: 弱学習器(決定木)の数
    - max_depth: 各弱学習器(決定木)の木の深さの上限
- scikit-learn の [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) を用いると、簡単にグリッドサーチ＆交差検証を行える
    - 交差検証の分割数は 5 とする（多くするほど時間がかかる）

In [7]:
# 5-分割交差検証を行うためのクラスをインスタンス化
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

# ハイパーパラメータの候補（合計 6*3=18 通り)
parameters = {
    'n_estimators': [50, 60, 70, 80, 90, 100], 
    'max_depth': [5, 10, 50]
}

# ランダムフォレストによる分類モデルをインスタンス化
rfc = RandomForestClassifier(criterion='entropy', random_state=1234)

# グリッドサーチを行うためのクラスをインスタンス化
# verbose: 出力文の細かさ（2 の場合、パラメータの値と処理時間を表示）
grid = GridSearchCV(
    rfc, param_grid=parameters, cv=kf, 
    scoring='accuracy', verbose=2
)

### 3-2. グリッドサーチの実行
- `time` モジュールを使って、ハイパーパラメータ探索にかかったトータルの処理時間を計測
- 作成者の環境では、7分25秒となった

In [8]:
# 計測開始
start_time = time.perf_counter() 

# 交差検証法を用いつつグリッドサーチを行う
# X_train は内部で訓練用、検証用に分割される
# 試行回数は 6*3*5 = 90 回
#（パラメータ候補*パラメータ候補*分割数）
grid.fit(X_train, y_train)

# 計測終了
end_time = time.perf_counter() 

# 経過時間を計算
elapsed_time = end_time - start_time
print(f'done in {elapsed_time} sec.')

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END .......................max_depth=5, n_estimators=50; total time=   1.0s
[CV] END .......................max_depth=5, n_estimators=50; total time=   1.2s
[CV] END .......................max_depth=5, n_estimators=50; total time=   1.1s
[CV] END .......................max_depth=5, n_estimators=50; total time=   1.0s
[CV] END .......................max_depth=5, n_estimators=50; total time=   1.0s
[CV] END .......................max_depth=5, n_estimators=60; total time=   1.2s
[CV] END .......................max_depth=5, n_estimators=60; total time=   1.2s
[CV] END .......................max_depth=5, n_estimators=60; total time=   1.2s
[CV] END .......................max_depth=5, n_estimators=60; total time=   1.1s
[CV] END .......................max_depth=5, n_estimators=60; total time=   1.2s
[CV] END .......................max_depth=5, n_estimators=70; total time=   1.4s
[CV] END .......................max_depth=5, n_e

- どのパラメータの組み合わせが一番よかったかは、`GridSearchCV` のメンバ変数 `best_params_` で確認できる
- 以下の結果から、max_depth = 50, n_estimators = 80 の時がベストであったことがわかる

In [9]:
# 最良のハイパーパラメータを確認
grid.best_params_

{'max_depth': 50, 'n_estimators': 80}

### 3-3. モデルの評価
- `GridSearchCV` の `predict` メソッドを用いる
- グリッドサーチで得られたベストなハイパーパラメータの組み合わせのモデルを用いてデータを予測
- インスタンス化の際に `refit = True` と設定していれば、再学習の必要はない（デフォルト値は `True`）


In [10]:
# 分類性能を表示する関数
def print_perf(clf, X, y):
    """
    clf: 分類器
    X: 学習データ
    y: 学習データの正解ラベル
    """
    y_pred = clf.predict(X)
    acc = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    
    print(f'正解率: {acc:.3}')
    print(f'Precision: {precision:.3}')
    print(f'Recall: {recall:.3}')
    print(f'F1: {f1:.3}\n')

In [11]:
# ベストなモデルを使って、テスト用データに対する予測を行う
# 学習用データに対する性能
print_perf(grid, X_train, y_train)

# テスト用データに対する性能
print_perf(grid, X_test, y_test)

正解率: 0.876
Precision: 0.915
Recall: 0.843
F1: 0.877

正解率: 0.781
Precision: 0.81
Recall: 0.74
F1: 0.774



DAY 2 模範回答では、テスト用データに対する accuracy や F1-score は 0.72 ほどであったので、改善されている